### Step 12.  Measure Summaries of Summaries Accuracy for Submissions and Comments

#### Import required libraries, load submissions and comments with summary

In [51]:
import torch
import bert_score

In [53]:
import pandas as pd

def extract_nan_rows(df, name):
    nan_rows = df[df['new_summary'].isna()]
    nan_rows['source'] = name
    return nan_rows

# Load DataFrames
df = pd.read_csv('tfcc_top_20_summaries_of_summaries.csv')
comments_df = pd.read_csv('tfcc_comments_summaries_of_summaries.csv')
pegasus_df = pd.read_csv('tfcc_top_20_summaries_of_summaries_pegasus.csv')
pegasus_comments_df = pd.read_csv('tfcc_comments_summaries_of_summaries_pegasus.csv')
cohere_df = pd.read_csv('tfcc_top_20_summaries_of_summaries.csv')
cohere_comments_df = pd.read_csv('tfcc_comments_summaries_of_summaries_cohere.csv')

# Extract rows with NaN values in the 'summary' column
nan_rows_df = pd.DataFrame()
nan_rows_df = nan_rows_df.append(extract_nan_rows(df, "tfcc_top_20_summaries_of_summaries"))
nan_rows_df = nan_rows_df.append(extract_nan_rows(comments_df, "tfcc_comments_summaries_of_summaries"))
nan_rows_df = nan_rows_df.append(extract_nan_rows(pegasus_df, "tfcc_top_20_summaries_of_summaries_pegasus"))
nan_rows_df = nan_rows_df.append(extract_nan_rows(pegasus_comments_df, "tfcc_comments_summaries_of_summaries_pegasus"))
nan_rows_df = nan_rows_df.append(extract_nan_rows(cohere_df, "tfcc_top_20_summaries_of_summaries"))
nan_rows_df = nan_rows_df.append(extract_nan_rows(cohere_comments_df, "tfcc_comments_summaries_of_summaries_cohere"))

# Remove rows with NaN values from the original DataFrames
df = df.dropna(subset=['new_summary'])
comments_df = comments_df.dropna(subset=['new_summary'])
pegasus_df = pegasus_df.dropna(subset=['new_summary'])
pegasus_comments_df = pegasus_comments_df.dropna(subset=['new_summary'])
cohere_df = cohere_df.dropna(subset=['new_summary'])
cohere_comments_df = cohere_comments_df.dropna(subset=['new_summary'])

# Print the DataFrame containing rows with NaN values in the 'summary' column
nan_rows_df.to_csv('summary_errors.csv', index=False)



/tmp/ipykernel_9188/2787036845.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nan_rows_df = nan_rows_df.append(extract_nan_rows(df, "tfcc_top_20_summaries_of_summaries"))
/tmp/ipykernel_9188/2787036845.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nan_rows_df = nan_rows_df.append(extract_nan_rows(comments_df, "tfcc_comments_summaries_of_summaries"))
/tmp/ipykernel_9188/2787036845.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nan_rows_df = nan_rows_df.append(extract_nan_rows(pegasus_df, "tfcc_top_20_summaries_of_summaries_pegasus"))
/tmp/ipykernel_9188/2787036845.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  na

In [54]:
nan_rows_df

,topic,summary,title,new_summary,theme,Count,topic_name,source,concatenated_summary


### Measure summary accuracy and drop all rows that are below 0.8 F1

In [55]:
# get the number of rows using the shape attribute
start_num_rows = df.shape[0]
pegasus_start_num_rows = pegasus_df.shape[0]
cohere_start_num_rows = cohere_df.shape[0]

In [56]:
print("OpenAI - Measuring accuracy on " +str(start_num_rows) + " rows.")
print("Pegasus - Measuring accuracy on " +str(pegasus_start_num_rows) + " rows.")
print("COhere - Measuring accuracy on " +str(cohere_start_num_rows) + " rows.")


OpenAI - Measuring accuracy on 98 rows.
Pegasus - Measuring accuracy on 205 rows.
COhere - Measuring accuracy on 98 rows.


#### Calculate accuracy for OpenAI submissions

In [57]:
import torch

# Your existing code
selftext = df['summary']
summary = df['new_summary']
P, R, F1 = bert_score.score(selftext.tolist(), summary.tolist(), lang='en', verbose=True)
df['bert_f1'] = F1.tolist()

# Free up GPU memory
del P, R, F1
torch.cuda.empty_cache()




Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 7.30 seconds, 13.43 sentences/sec


#### Calculate accuracy for Pegasus submissions

In [58]:
selftext = pegasus_df['summary']
summary = pegasus_df['new_summary']

# Compute the BERTScore for each pair of text
P, R, F1 = bert_score.score(selftext.tolist(), summary.tolist(), lang='en', verbose=True)
pegasus_df['bert_f1'] = F1.tolist()

# Free up GPU memory
del P, R, F1
torch.cuda.empty_cache()


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 13.67 seconds, 14.99 sentences/sec


#### Calculate accuracy for Cohere submissions

In [59]:
selftext = cohere_df['summary']
summary = cohere_df['new_summary']

# Compute the BERTScore for each pair of text
P, R, F1 = bert_score.score(selftext.tolist(), summary.tolist(), lang='en', verbose=True)
cohere_df['bert_f1'] = F1.tolist()

# Free up GPU memory
del P, R, F1
torch.cuda.empty_cache()


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 7.42 seconds, 13.21 sentences/sec


#### Display average accuracy score for OpenAI, Pegasus, and Cohere

In [60]:
from IPython.display import display, HTML

# Assuming you have already loaded the dataframes: df, pegasus_df, and cohere_df

# Calculate the average F1 scores
df_avg_f1 = df['bert_f1'].mean()
pegasus_avg_f1 = pegasus_df['bert_f1'].mean()
cohere_avg_f1 = cohere_df['bert_f1'].mean()

# Create a dictionary to store the results
data = {
    'Dataframe': ['df', 'pegasus_df', 'cohere_df'],
    'Average F1': [df_avg_f1, pegasus_avg_f1, cohere_avg_f1]
}

# Create a new dataframe to display the results
results_df = pd.DataFrame(data)

# Display the results in a simple table
display(HTML(results_df.to_html(index=False)))


Dataframe,Average F1
df,0.847731
pegasus_df,0.895354
cohere_df,0.847731


#### Print results for OpenAI test

In [61]:
df2 = df[df['bert_f1'] < 0.8] 
df = df[df['bert_f1'] >= 0.8]

end_num_rows = df.shape[0]

removed_num = df2.shape[0]

print(str(end_num_rows) + " rows were >= 0.8 F1 and " + str(removed_num) + " were below 0.8 F1 and have been removed.")


98 rows were >= 0.8 F1 and 0 were below 0.8 F1 and have been removed.


In [62]:
df.to_csv('tfcc_submissions_summaries_of_summaries_with_accuracy.csv', index=False)

#### Print results for Pegasus test

In [63]:
pegasus_df2 = pegasus_df[pegasus_df['bert_f1'] < 0.8] 
pegasus_df = pegasus_df[pegasus_df['bert_f1'] >= 0.8]

end_num_rows = pegasus_df.shape[0]

removed_num = pegasus_df2.shape[0]

print(str(end_num_rows) + " rows were >= 0.8 F1 and " + str(removed_num) + " were below 0.8 F1 and have been removed.")

205 rows were >= 0.8 F1 and 0 were below 0.8 F1 and have been removed.


In [64]:
pegasus_df.to_csv('tfcc_submissions_summaries_of_summaries_pegasus_with_accuracy.csv', index=False)

#### Print results for Cohere test

In [65]:
temp_df2 = cohere_df[cohere_df['bert_f1'] < 0.8] 
temp_df = cohere_df[cohere_df['bert_f1'] >= 0.8]

end_num_rows = temp_df.shape[0]

removed_num = temp_df2.shape[0]

print(str(end_num_rows) + " rows were >= 0.8 F1 and " + str(removed_num) + " were below 0.8 F1.")

98 rows were >= 0.8 F1 and 0 were below 0.8 F1.


In [16]:
cohere_df.to_csv('tfcc_submissions_summaries_of_summaries_cohere_with_accuracy.csv', index=False)

### Repeat accuracy measurement process for comments.  

In [66]:
# get the number of rows using the shape attribute
start_num_rows = comments_df.shape[0]
print("OpenAI Measuring accuracy on " +str(start_num_rows) + " rows.")

pegasus_start_num_rows = pegasus_comments_df.shape[0]
print("Pegasus Measuring accuracy on " +str(pegasus_start_num_rows) + " rows.")

cohere_start_num_rows = cohere_comments_df.shape[0]
print("Cohere Measuring accuracy on " +str(cohere_start_num_rows) + " rows.")

OpenAI Measuring accuracy on 32 rows.
Pegasus Measuring accuracy on 20 rows.
Cohere Measuring accuracy on 20 rows.


#### Measure OpenAI comments summaries accuracy

In [67]:
# Select the columns to compare
selftext = comments_df['summary']
summary = comments_df['new_summary']

# Compute the BERTScore for each pair of text
P, R, F1 = bert_score.score(selftext.tolist(), summary.tolist(), lang='en', verbose=True)
comments_df['bert_f1'] = F1.tolist()

# Free up GPU memory
del P, R, F1
torch.cuda.empty_cache()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.98 seconds, 10.74 sentences/sec


#### Measure Pegasus comments summaries accuracy

In [68]:
# Select the columns to compare
selftext = pegasus_comments_df['summary']
summary = pegasus_comments_df['new_summary']

# Compute the BERTScore for each pair of text
P, R, F1 = bert_score.score(selftext.tolist(), summary.tolist(), lang='en', verbose=True)
pegasus_comments_df['bert_f1'] = F1.tolist()

# Free up GPU memory
del P, R, F1
torch.cuda.empty_cache()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.52 seconds, 13.12 sentences/sec


#### Measure Cohere comments summaries accuracy

In [48]:
cohere_comments_df

,topic,concatenated_summary,new_summary
0,0.0,I work in a call center. I got a call from a v...,I work in a call center. I got a call from a v...
1,1.0,I used to work for the mental health branch of...,I had a mother call me one time to request we ...
2,2.0,I had to go back and reread that. You're givin...,OP is a salesperson who sold a TV to a custome...
3,3.0,I work for a different variety of insurance an...,I'm sorry but I'm not going to give you a clai...
4,4.0,A woman called a dental office to ask for her ...,I work in a call center. I deal with a lot of ...
5,5.0,A woman in the US has been filmed berating a c...,A woman in the US has been filmed berating a c...
6,6.0,I had a similar situation when doing customer ...,I'm an ex-call center worker. I worked for a m...
7,7.0,I quit my job after 20 years and made over 27 ...,I worked at a call center. We would drink on t...
8,8.0,I don't blame you for being upset. I would be ...,I'm so sorry you had to go through that. Ive h...
9,9.0,I was in retail for a decade. I was in a call ...,I've been in a call centre for 5 years. I cry ...


In [69]:
# Select the columns to compare
selftext = cohere_comments_df['concatenated_summary']
summary = cohere_comments_df['new_summary']

# Compute the BERTScore for each pair of text
P, R, F1 = bert_score.score(selftext.tolist(), summary.tolist(), lang='en', verbose=True)
cohere_comments_df['bert_f1'] = F1.tolist()

# Free up GPU memory
del P, R, F1
torch.cuda.empty_cache()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.84 seconds, 10.87 sentences/sec


#### Display average accuracy score for OpenAI, Pegasus, and Cohere

In [70]:
# Calculate the average F1 scores
df_avg_f1 = comments_df['bert_f1'].mean()
pegasus_avg_f1 = pegasus_comments_df['bert_f1'].mean()
cohere_avg_f1 = cohere_comments_df['bert_f1'].mean()

# Create a dictionary to store the results
data = {
    'Dataframe': ['df', 'pegasus_df', 'cohere_df'],
    'Average F1': [df_avg_f1, pegasus_avg_f1, cohere_avg_f1]
}

# Create a new dataframe to display the results
results_df = pd.DataFrame(data)

# Display the results in a simple table
display(HTML(results_df.to_html(index=False)))

Dataframe,Average F1
df,0.875641
pegasus_df,0.925182
cohere_df,0.877726


#### Print OpenAI results

In [71]:
temp_df2 = comments_df[comments_df['bert_f1'] < 0.8] 
temp_df = comments_df[comments_df['bert_f1'] >= 0.8]

end_num_rows = temp_df.shape[0]

removed_num = temp_df2.shape[0]

print(str(end_num_rows) + " rows were >= 0.8 F1 and " + str(removed_num) + " were below 0.8 F1.")


32 rows were >= 0.8 F1 and 0 were below 0.8 F1.


#### Print Pegasus results

In [72]:
temp_df2 = pegasus_comments_df[pegasus_comments_df['bert_f1'] < 0.8] 
temp_df = pegasus_comments_df[pegasus_comments_df['bert_f1'] >= 0.8]

end_num_rows = temp_df.shape[0]

removed_num = temp_df2.shape[0]

print(str(end_num_rows) + " rows were >= 0.8 F1 and " + str(removed_num) + " were below 0.8 F1.")


20 rows were >= 0.8 F1 and 0 were below 0.8 F1.


#### Print Cohere results

In [73]:
temp_df2 = cohere_comments_df[cohere_comments_df['bert_f1'] < 0.8] 
temp_df = cohere_comments_df[cohere_comments_df['bert_f1'] >= 0.8]

end_num_rows = temp_df.shape[0]

removed_num = temp_df2.shape[0]

print(str(end_num_rows) + " rows were >= 0.8 F1 and " + str(removed_num) + " were below 0.8 F1.")

18 rows were >= 0.8 F1 and 2 were below 0.8 F1.


#### Double check that we still have comments for every top.  Count should = 20

OpenAI:

In [74]:
unique_count = comments_df['topic'].nunique()

print(unique_count)

20


Pegasus:

In [75]:
unique_count = pegasus_comments_df['topic'].nunique()

print(unique_count)

20


Cohere:

In [76]:
unique_count = cohere_comments_df['topic'].nunique()

print(unique_count)

20


#### Export comments with accuracy to CSV

In [77]:
comments_df.to_csv('tfcc_comments_summaries_of_summaries_with_accuracy.csv', index=False)
pegasus_comments_df.to_csv('tfcc_pegasus_comments_summaries_of_summaries_with_accuracy.csv', index=False)
cohere_comments_df.to_csv('tfcc_cohere_comments_summaries_of_summaries_with_accuracy.csv', index=False)

#### Generate accuracy report

In [78]:
import pandas as pd

In [79]:
openai_df = pd.read_csv('tfcc_submissions_summaries_of_summaries_with_accuracy.csv')
pegasus_df = pd.read_csv('tfcc_submissions_summaries_of_summaries_pegasus_with_accuracy.csv')
cohere_df = pd.read_csv('tfcc_submissions_summaries_of_summaries_cohere_with_accuracy.csv')

openai_comments_df = pd.read_csv('tfcc_comments_summaries_of_summaries_with_accuracy.csv')
pegasus_comments_df = pd.read_csv('tfcc_pegasus_comments_summaries_of_summaries_with_accuracy.csv')
cohere_comments_df = pd.read_csv('tfcc_cohere_comments_summaries_of_summaries_with_accuracy.csv')


topics = pd.read_csv('tfcc_top20_topics_with_sentiment_and_comments_sentiment.csv')

In [80]:
topics.rename(columns={'Topic': 'topic'}, inplace=True)


In [82]:
import pandas as pd

# Group by 'topic' and calculate the mean F1 score for each model
openai_average_f1 = openai_df.groupby('topic')['bert_f1'].mean().reset_index().rename(columns={'bert_f1': 'openai_bert_f1'})
pegasus_average_f1 = pegasus_df.groupby('topic')['bert_f1'].mean().reset_index().rename(columns={'bert_f1': 'pegasus_bert_f1'})
cohere_average_f1 = cohere_df.groupby('topic')['bert_f1'].mean().reset_index().rename(columns={'bert_f1': 'cohere_bert_f1'})

# Merge the topics dataframe with the average F1 scores dataframes on the 'topic' column
topics_with_f1 = topics.merge(openai_average_f1, on='topic', how='left')
topics_with_f1 = topics_with_f1.merge(pegasus_average_f1, on='topic', how='left')
topics_with_f1 = topics_with_f1.merge(cohere_average_f1, on='topic', how='left')


In [83]:
topics_with_f1

,topic,Count,Name,avg_pos_sentiment,avg_neg_sentiment,avg_comments_pos_sentiment,avg_comments_neg_sentiment,openai_bert_f1,pegasus_bert_f1,cohere_bert_f1
0,0,796,0_like_help_name_say,0.161216,0.838784,0.196144,0.803856,0.850991,0.891258,0.850991
1,1,591,1_card_bank_credit_fraud,0.113983,0.886017,0.148113,0.851887,0.841732,0.900285,0.841732
2,2,457,2_delivery_shipping_store_email,0.122277,0.877723,0.163302,0.836698,0.843909,0.895476,0.843909
3,3,283,3_insurance_car_claims_coverage,0.103466,0.896534,0.117970,0.882030,0.847239,0.895246,0.847239
4,4,243,4_patient_doctor_clinic_medical,0.139520,0.860480,0.159091,0.840909,0.845988,0.894306,0.845988
5,5,201,5_tow_roadside_truck_assistance,0.159511,0.840489,0.139324,0.860676,0.847994,0.898121,0.847994
6,6,188,6_bill_phones_service_data,0.133500,0.866500,0.103897,0.896103,0.852856,0.895266,0.852856
7,7,108,7_job_feel_anxiety_work,0.154886,0.845114,0.290429,0.709571,0.845393,0.891779,0.845393
8,8,106,8_english_spanish_speak_language,0.154176,0.845824,0.208100,0.791900,0.847847,0.893386,0.847847
9,9,101,9_job_interview_role_experience,0.374143,0.625857,0.343623,0.656377,0.842245,0.889179,0.842245


In [84]:
topics_with_f1 = topics_with_f1.drop(columns=['Count', 'avg_pos_sentiment', 'avg_neg_sentiment', 'avg_comments_pos_sentiment', 'avg_comments_neg_sentiment'])


In [85]:
topics_with_f1

,topic,Name,openai_bert_f1,pegasus_bert_f1,cohere_bert_f1
0,0,0_like_help_name_say,0.850991,0.891258,0.850991
1,1,1_card_bank_credit_fraud,0.841732,0.900285,0.841732
2,2,2_delivery_shipping_store_email,0.843909,0.895476,0.843909
3,3,3_insurance_car_claims_coverage,0.847239,0.895246,0.847239
4,4,4_patient_doctor_clinic_medical,0.845988,0.894306,0.845988
5,5,5_tow_roadside_truck_assistance,0.847994,0.898121,0.847994
6,6,6_bill_phones_service_data,0.852856,0.895266,0.852856
7,7,7_job_feel_anxiety_work,0.845393,0.891779,0.845393
8,8,8_english_spanish_speak_language,0.847847,0.893386,0.847847
9,9,9_job_interview_role_experience,0.842245,0.889179,0.842245


In [86]:
# Group by 'topic' and calculate the mean F1 score for each model
openai_average_f1 = openai_comments_df.groupby('topic')['bert_f1'].mean().reset_index().rename(columns={'bert_f1': 'openai_bert_f1'})
pegasus_average_f1 = pegasus_comments_df.groupby('topic')['bert_f1'].mean().reset_index().rename(columns={'bert_f1': 'pegasus_bert_f1'})
cohere_average_f1 = cohere_comments_df.groupby('topic')['bert_f1'].mean().reset_index().rename(columns={'bert_f1': 'cohere_bert_f1'})

# Merge the topics dataframe with the average F1 scores dataframes on the 'topic' column
comments_topics_with_f1 = topics.merge(openai_average_f1, on='topic', how='left')
comments_topics_with_f1 = comments_topics_with_f1.merge(pegasus_average_f1, on='topic', how='left')
comments_topics_with_f1 = comments_topics_with_f1.merge(cohere_average_f1, on='topic', how='left')

In [87]:
comments_topics_with_f1 = comments_topics_with_f1.drop(columns=['Count','avg_pos_sentiment', 'avg_neg_sentiment', 'avg_comments_pos_sentiment', 'avg_comments_neg_sentiment'])


In [88]:
comments_topics_with_f1

,topic,Name,openai_bert_f1,pegasus_bert_f1,cohere_bert_f1
0,0,0_like_help_name_say,0.893958,0.924967,0.901824
1,1,1_card_bank_credit_fraud,0.884805,0.939266,0.849363
2,2,2_delivery_shipping_store_email,0.876737,0.911328,0.810306
3,3,3_insurance_car_claims_coverage,0.889987,0.949932,0.790009
4,4,4_patient_doctor_clinic_medical,0.863482,0.933419,0.789450
5,5,5_tow_roadside_truck_assistance,0.859110,0.914904,0.893871
6,6,6_bill_phones_service_data,0.877826,0.911036,0.802446
7,7,7_job_feel_anxiety_work,0.890982,0.910531,0.931457
8,8,8_english_spanish_speak_language,0.842684,0.920296,0.870770
9,9,9_job_interview_role_experience,0.859843,0.903461,0.879219
